## Week 3 Data Generator With Opensource Models
# Generate synthetic data for Pizza cusromers within Nairobi 

In [ ]:
!pip install requests pandas ipywidgets gradio

In [ ]:
import pandas as pd
import gradio as gr
from huggingface_hub import InferenceClient
import random
import os
from dotenv import load_dotenv


In [ ]:
#Load API Key

load_dotenv(override=True)
HF_API_KEY = os.getenv('HF_TOKEN')

In [ ]:

# Define available models with correct Hugging Face model IDs
MODELS = {
    "Mistral-7B": "mistralai/Mistral-7B-Instruct-v0.2",
    "Llama-2-7B": "meta-llama/Llama-2-7b-chat-hf",
    "Phi-2": "microsoft/phi-2",
    "GPT-2": "gpt2"
}

# Nairobi branches
BRANCHES = ["Westlands", "Karen", "Kilimani", "CBD", "Parklands"]

# Global variable to store generated data
generated_df = None

def generate_feedback_data(model_name, num_records):
    """Generate synthetic pizza feedback data using selected AI model"""
    global generated_df
    
    try:
        # Initialize the Hugging Face Inference Client
        model_id = MODELS[model_name]
        client = InferenceClient(model=model_id, token=None)  # Add your HF token if needed
        
        feedback_data = []
        
        for i in range(num_records):
            # Random branch
            branch = random.choice(BRANCHES)
            
            # Generate feedback using the AI model
            prompt = f"Generate a brief customer feedback comment about a pizza order from {branch} branch in Nairobi. Make it realistic and varied (positive, negative, or neutral). Keep it under 30 words."
            
            try:
                response = client.text_generation(
                    prompt,
                    max_new_tokens=50,
                    temperature=0.8
                )
                feedback = response.strip()
            except Exception as e:
                # Fallback to template-based generation if API fails
                feedback = generate_fallback_feedback(branch)
            
            # Generate other fields
            record = {
                "Customer_ID": f"CUST{1000 + i}",
                "Branch": branch,
                "Rating": random.randint(1, 5),
                "Order_Type": random.choice(["Delivery", "Dine-in", "Takeaway"]),
                "Feedback": feedback,
                "Date": f"2024-{random.randint(1, 12):02d}-{random.randint(1, 28):02d}"
            }
            
            feedback_data.append(record)
        
        # Create DataFrame
        generated_df = pd.DataFrame(feedback_data)
        
        return generated_df, f"✓ Successfully generated {num_records} records using {model_name}"
    
    except Exception as e:
        return pd.DataFrame(), f"✗ Error: {str(e)}"

def generate_fallback_feedback(branch):
    """Fallback feedback generator if API fails"""
    templates = [
        f"Great pizza from {branch}! Quick delivery and hot food.",
        f"Pizza was cold when it arrived at {branch}. Disappointed.",
        f"Excellent service at {branch} branch. Will order again!",
        f"Average experience. Pizza was okay but nothing special.",
        f"Long wait time at {branch} but the pizza was worth it.",
    ]
    return random.choice(templates)

def download_csv():
    """Save generated data as CSV"""
    global generated_df
    if generated_df is not None:
        generated_df.to_csv('pizza_feedback_data.csv', index=False)
        return "CSV downloaded!"
    return "No data to download"

def download_json():
    """Save generated data as JSON"""
    global generated_df
    if generated_df is not None:
        generated_df.to_json('pizza_feedback_data.json', orient='records', indent=2)
        return "JSON downloaded!"
    return "No data to download"

# Create Gradio interface
with gr.Blocks(title="Pizza Feedback Data Generator") as demo:
    gr.Markdown("""
    # 🍕 Pizza Feedback Data Generator
    Generate synthetic customer feedback for Nairobi pizza branches using AI models
    """)
    
    with gr.Row():
        with gr.Column():
            model_selector = gr.Radio(
                choices=list(MODELS.keys()),
                label="Select AI Model",
                value=list(MODELS.keys())[0]
            )
            
            num_records_slider = gr.Slider(
                minimum=1,
                maximum=50,
                value=10,
                step=1,
                label="Number of Records"
            )
            
            generate_btn = gr.Button("Generate Feedback Data", variant="primary")
    
    with gr.Row():
        status_output = gr.Textbox(label="Status", interactive=False)
    
    with gr.Row():
        dataframe_output = gr.Dataframe(
            label="Generated Feedback Data",
            interactive=False
        )
    
    with gr.Row():
        csv_btn = gr.Button("Download CSV")
        json_btn = gr.Button("Download JSON")
    
    # Event handlers
    generate_btn.click(
        fn=generate_feedback_data,
        inputs=[model_selector, num_records_slider],
        outputs=[dataframe_output, status_output]
    )
    
    csv_btn.click(
        fn=download_csv,
        outputs=status_output
    )
    
    json_btn.click(
        fn=download_json,
        outputs=status_output
    )

# Launch the interface
demo.launch()